### Annotation instructions:

1. Rotate the image to find a position, where the filament is clearly visible
2. Draw a line over the filament, by holding "Control" and dragging the mouse
3. Rotate the image to view the filament from another angle and repeat step 2
4. Rotate the image again: this will trigger calculating the filament position
5. Repeat steps 1-4 for other filaments

In [1]:
import napari

from lib.lib import *

In [2]:
output_fn = 'annotations.csv'

In [3]:
img = create_random_lines(100, 10)
viewer = napari.view_image(img, ndisplay=3)
an_layer = add_annotation_layer(viewer)

near_points = []
far_points = []
polygons = []

@an_layer.mouse_drag_callbacks.append
def on_drag(layer, event):
    yield
    while event.type == 'mouse_move':
        near_point, far_point = layer.get_ray_intersections(
            event.position,
            event.view_direction,
            event.dims_displayed
        )
        if 'Control' in event.modifiers:
            if (near_point is not None) and (far_point is not None):
                near_points.append(near_point)
                far_points.append(far_point)
            
            if len(near_points) > 3:
                layer = draw_polygon(layer, near_points, far_points)
                
        else:
            if len(near_points) > 0:
                polygons.append([near_points.copy(), far_points.copy()])
            near_points.clear()
            far_points.clear()
            if len(polygons) >= 2:
                npt1 = polygons[0][0]
                npt2 = polygons[1][0]
                fpt1 = polygons[0][1]
                fpt2 = polygons[1][1]
                mt = compute_polygon_intersection(npt1, npt2, fpt1, fpt2)
                mt = sort_points(mt)
                layer.selected_data = set(range(layer.nshapes - 2, layer.nshapes))
                layer.remove_selected()
                layer.add(mt, shape_type='path', edge_color='green', edge_width=1)
                polygons[0] = None
                polygons[1] = None
                polygons.clear()
                annotation_to_pandas(layer.data[1:]).to_csv(output_fn, index=False)
            
        yield
